In [127]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [128]:
venta_productos = [
    ('20201010', 1, 100),
    ('20191010', 1, 100),
    ('20191011', 2, 200),
    ('20191012', 3, 300),
    ('20201010', 3, 600),
    ('20201010', 3, 600),
    ('20191013', 2, 200),
    ('20201014', 2, 800),
    ('20191017', 2, 200),
    ('20181013', 4, 400),
    ('20181013', 5, 400),
    ('20201013', 5, 400)
]
detalles_productos = [
    ( 1, 'hago cosas', 'categoria1'),
    ( 2, 'hago cosas', 'categoria2'),
    ( 3, 'hago cosas', 'categoria3'),
    ( 2, 'hago cosas', 'categoria2'),
    ( 2, 'hago cosas', 'categoria2'),
    ( 3, 'hago cosas', 'categoria3'),
    ( 3, 'hago cosas', 'categoria3'),
    ( 4, 'no hago cosas', 'categoria4'),
    ( 5, 'no hago cosas', 'categoria4')
]

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

venta_productos = sc.parallelize(venta_productos)
detalles_productos = sc.parallelize(detalles_productos)

2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de producto, precio de venta) y en otro RDD detalle de los productos con (código de producto, descripción del producto, categoría). Se pide resolver utilizando PySpark:

Cuál es el producto más vendido.



In [129]:
ventas_por_producto = venta_productos.map(lambda x :(x[1], 1)).reduceByKey(lambda x,y: (x+y)).cache()
ventas_por_producto.reduce(lambda x,y:(x if x[1]>y[1] else y))

(2, 4)

Cuál es la categoría de productos más vendida.



In [130]:
ventas_por_producto.take(5)

[(4, 1), (1, 2), (5, 2), (2, 4), (3, 3)]

In [131]:
pene = detalles_productos.map(lambda x: (x[0],x[2])).distinct().join(ventas_por_producto).reduce(lambda x,y:(x if x[1][1]>y[1][1] else y))
pene[1][0]

'categoria2'

Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)


In [132]:
ventas_por_producto.join(detalles_productos).distinct().map(lambda x: (x[0],(x[1][1],x[1][0]))).takeOrdered(5,lambda x: -x[1][1])

[(2, ('hago cosas', 4)),
 (3, ('hago cosas', 3)),
 (5, ('no hago cosas', 2)),
 (1, ('hago cosas', 2)),
 (4, ('no hago cosas', 1))]

Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 3 ventas en el último año.


In [133]:
ventas_2020 = venta_productos.filter(lambda x : x[0]>='20200101').cache()
ventas_2019 = venta_productos.filter(lambda x : x[0]>='20190101' and x[0]<'20200101').cache()

In [134]:
ventas_2019.collect()

[('20191010', 1, 100),
 ('20191011', 2, 200),
 ('20191012', 3, 300),
 ('20191013', 2, 200),
 ('20191017', 2, 200)]

In [135]:
ventas_2020 = ventas_2020.map(lambda x: (x[1],(x[2]))).cache()
ventas_2019 = ventas_2019.map(lambda x: (x[1],(x[2]))).cache()
print(ventas_2020.collect())
print(ventas_2019.collect())

[(1, 100), (3, 600), (3, 600), (2, 800), (5, 400)]
[(1, 100), (2, 200), (3, 300), (2, 200), (2, 200)]


In [136]:
ventas_2020.join(ventas_2019).distinct()\
.map(lambda x: (x[0], (x[1][0]-x[1][1])))\
.reduce(lambda x,y: x if x[1]>y[1] else y)[0]

2

Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.